# LLM Optimization Modelling Experiment

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [131]:
problem = '''We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number. 

We want to maximize the daily profit of this alkylation process. 
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup. 

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas. 
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed. 
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate. 
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup.'''

## 2. Generate the mathematical model

In [132]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [133]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Alkylation Process

**Parameters:**

* $p$: Revenue per barrel of alkylate per octane number
* $c_o$: Cost per barrel of olefin feed
* $c_i$: Cost per barrel of isobutane makeup
* $c_r$: Cost per barrel of isobutane recycle
* $c_a$: Cost per thousand pounds of acid addition

**Decision Variables:**

* $OF$: Olefin feed (barrels per day)
* $IR$: Isobutane recycle (barrels per day)
* $AAR$: Acid addition rate (thousands of pounds per day)
* $AY$: Alkylate yield (barrels per day)
* $IM$: Isobutane makeup (barrels per day)
* $AS$: Acid strength (weight percent)
* $MON$: Motor octane number
* $IOR$: External isobutane-to-olefin ratio
* $ADF$: Acid dilution factor
* $F4$: F-4 performance number

**Objective Function:**

Maximize daily profit:

$$
\max Z = p \cdot AY \cdot MON - (c_o \cdot OF + c_i \cdot IM + c_r \cdot IR + c_a \cdot AAR)
$$

**Constraints:**

1. **Alkylate yield regression:**
   - $AY = OF \cdot (1.12 + 0.13167 \cdot IOR - 0.00667 \cdot IOR^2) \cdot (1 + \epsilon_1)$, where $|\epsilon_1| \leq 0.02$ 

2. **Motor octane number regression:**
   - $MON = 86.35 + 1.098 \cdot IOR - 0.038 \cdot IOR^2 + 0.325 \cdot (AS-89) \cdot (1 + \epsilon_2)$, where $|\epsilon_2| \leq 0.01$

3. **Acid dilution factor regression:**
   - $ADF = 35.82 - 0.222 \cdot F4 \cdot (1 + \epsilon_3)$, where $|\epsilon_3| \leq 0.05$

4. **F-4 performance number regression:**
   - $F4 = -133 + 3 \cdot MON \cdot (1 + \epsilon_4)$, where $|\epsilon_4| \leq 0.1$

5. **External isobutane-to-olefin ratio:**
   - $IOR = (IR + IM) / OF$

6. **Acid strength:**
   - $AS = (93000 \cdot AAR) / (AAR \cdot ADF + 1000 \cdot AAR)$

7. **Alkylate yield balance:**
   - $1.22 \cdot AY = OF + IM$

8. **Variable bounds:**
   - $OF_{lb} \leq OF \leq OF_{ub}$
   - $IR_{lb} \leq IR \leq IR_{ub}$
   - $AAR_{lb} \leq AAR \leq AAR_{ub}$
   - $AY_{lb} \leq AY \leq AY_{ub}$
   - $IM_{lb} \leq IM \leq IM_{ub}$
   - $AS_{lb} \leq AS \leq AS_{ub}$
   - $MON_{lb} \leq MON \leq MON_{ub}$
   - $IOR_{lb} \leq IOR \leq IOR_{ub}$
   - $ADF_{lb} \leq ADF \leq ADF_{ub}$
   - $F4_{lb} \leq F4 \leq F4_{ub}$

This model seeks to maximize the daily profit of the alkylation process by optimizing the values of ten decision variables while considering the uncertainties in regression estimations and adhering to the inherent constraints of the chemical process. 


## 3. Generate the pyomo code

In [134]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [135]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data if available)
data = {
    'p': 2.5,  # Revenue per barrel of alkylate per octane number
    'c_o': 50,  # Cost per barrel of olefin feed
    'c_i': 45,  # Cost per barrel of isobutane makeup
    'c_r': 40,  # Cost per barrel of isobutane recycle
    'c_a': 0.05,  # Cost per thousand pounds of acid addition
    'OF_lb': 1000,  # Lower bound of olefin feed (barrels per day)
    'OF_ub': 5000,  # Upper bound of olefin feed (barrels per day)
    'IR_lb': 500,   # Lower bound of isobutane recycle (barrels per day)
    'IR_ub': 4000,  # Upper bound of isobutane recycle (barrels per day)
    'AAR_lb': 10,   # Lower bound of acid addition rate (thousands of pounds per day)
    'AAR_ub': 100,  # Upper bound of acid addition rate (thousands of pounds per day)
    'AY_lb': 1500,  # Lower bound of alkylate yield (barrels per day)
    'AY_ub': 7000,  # Upper bound of alkylate yield (barrels per day)
    'IM_lb': 0,     # Lower bound of isobutane makeup (barrels per day)
    'IM_ub': 2000,  # Upper bound of isobutane makeup (barrels per day)
    'AS_lb': 88,    # Lower bound of acid strength (weight percent)
    'AS_ub': 92,    # Upper bound of acid strength (weight percent)
    'MON_lb': 88,    # Lower bound of Motor octane number
    'MON_ub': 95,    # Upper bound of Motor octane number
    'IOR_lb': 3,     # Lower bound of External isobutane-to-olefin ratio
    'IOR_ub': 6,     # Upper bound of External isobutane-to-olefin ratio
    'ADF_lb': 20,    # Lower bound of Acid dilution factor
    'ADF_ub': 50,    # Upper bound of Acid dilution factor
    'F4_lb': 100,   # Lower bound of F-4 performance number
    'F4_ub': 300,   # Upper bound of F-4 performance number
}

# Model creation
model = pyo.ConcreteModel()

# Decision variables
model.OF = pyo.Var(bounds=(data['OF_lb'], data['OF_ub']))
model.IR = pyo.Var(bounds=(data['IR_lb'], data['IR_ub']))
model.AAR = pyo.Var(bounds=(data['AAR_lb'], data['AAR_ub']))
model.AY = pyo.Var(bounds=(data['AY_lb'], data['AY_ub']))
model.IM = pyo.Var(bounds=(data['IM_lb'], data['IM_ub']))
model.AS = pyo.Var(bounds=(data['AS_lb'], data['AS_ub']))
model.MON = pyo.Var(bounds=(data['MON_lb'], data['MON_ub']))
model.IOR = pyo.Var(bounds=(data['IOR_lb'], data['IOR_ub']))
model.ADF = pyo.Var(bounds=(data['ADF_lb'], data['ADF_ub']))
model.F4 = pyo.Var(bounds=(data['F4_lb'], data['F4_ub']))

# Objective function
model.profit = pyo.Objective(expr=data['p'] * model.AY * model.MON - (data['c_o'] * model.OF + data['c_i'] * model.IM + data['c_r'] * model.IR + data['c_a'] * model.AAR), sense=pyo.maximize)

# Constraints
model.alkylate_yield = pyo.Constraint(expr=model.AY == model.OF * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * (1 + model.alkylate_yield_epsilon))
model.alkylate_yield_epsilon = pyo.Var(bounds=(-0.02, 0.02))

model.motor_octane_number = pyo.Constraint(expr=model.MON == 86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * (model.AS - 89) * (1 + model.motor_octane_number_epsilon))
model.motor_octane_number_epsilon = pyo.Var(bounds=(-0.01, 0.01))

model.acid_dilution_factor = pyo.Constraint(expr=model.ADF == 35.82 - 0.222 * model.F4 * (1 + model.acid_dilution_factor_epsilon))
model.acid_dilution_factor_epsilon = pyo.Var(bounds=(-0.05, 0.05))

model.f4_performance_number = pyo.Constraint(expr=model.F4 == -133 + 3 * model.MON * (1 + model.f4_performance_number_epsilon))
model.f4_performance_number_epsilon = pyo.Var(bounds=(-0.1, 0.1))

model.external_isobutane_to_olefin_ratio = pyo.Constraint(expr=model.IOR == (model.IR + model.IM) / model.OF)

model.acid_strength = pyo.Constraint(expr=model.AS == (93000 * model.AAR) / (model.AAR * model.ADF + 1000 * model.AAR))

model.alkylate_yield_balance = pyo.Constraint(expr=1.22 * model.AY == model.OF + model.IM)

# Solve the model
solver = pyo.SolverFactory('ipopt')
results = solver.solve(model)

# Print results
print(results)
print("Profit:", model.profit())
print("Olefin feed:", model.OF())
print("Isobutane recycle:", model.IR())
print("Acid addition rate:", model.AAR())
print("Alkylate yield:", model.AY())
print("Isobutane makeup:", model.IM())
print("Acid strength:", model.AS())
print("Motor octane number:", model.MON())
print("External isobutane-to-olefin ratio:", model.IOR())
print("Acid dilution factor:", model.ADF())
print("F-4 performance number:", model.F4())
```

This code defines the model, variables, objective function, and constraints as described in the mathematical formulation. The sample data is used for parameter values and variable bounds, which you should replace with your actual data. After solving the model, the code prints the solver status and the optimal values for the decision variables and the objective function. 


## 4. Input problem data and try running the generated code

In [136]:
import pyomo.environ as pyo

# Sample data (replace with actual data if available)
data = {
    'p': 2.5,  # Revenue per barrel of alkylate per octane number
    'c_o': 50,  # Cost per barrel of olefin feed
    'c_i': 45,  # Cost per barrel of isobutane makeup
    'c_r': 40,  # Cost per barrel of isobutane recycle
    'c_a': 0.05,  # Cost per thousand pounds of acid addition
    'OF_lb': 1000,  # Lower bound of olefin feed (barrels per day)
    'OF_ub': 5000,  # Upper bound of olefin feed (barrels per day)
    'IR_lb': 500,   # Lower bound of isobutane recycle (barrels per day)
    'IR_ub': 4000,  # Upper bound of isobutane recycle (barrels per day)
    'AAR_lb': 10,   # Lower bound of acid addition rate (thousands of pounds per day)
    'AAR_ub': 100,  # Upper bound of acid addition rate (thousands of pounds per day)
    'AY_lb': 1500,  # Lower bound of alkylate yield (barrels per day)
    'AY_ub': 7000,  # Upper bound of alkylate yield (barrels per day)
    'IM_lb': 0,     # Lower bound of isobutane makeup (barrels per day)
    'IM_ub': 2000,  # Upper bound of isobutane makeup (barrels per day)
    'AS_lb': 88,    # Lower bound of acid strength (weight percent)
    'AS_ub': 92,    # Upper bound of acid strength (weight percent)
    'MON_lb': 88,    # Lower bound of Motor octane number
    'MON_ub': 95,    # Upper bound of Motor octane number
    'IOR_lb': 3,     # Lower bound of External isobutane-to-olefin ratio
    'IOR_ub': 6,     # Upper bound of External isobutane-to-olefin ratio
    'ADF_lb': 20,    # Lower bound of Acid dilution factor
    'ADF_ub': 50,    # Upper bound of Acid dilution factor
    'F4_lb': 100,   # Lower bound of F-4 performance number
    'F4_ub': 300,   # Upper bound of F-4 performance number
}

# Model creation
model = pyo.ConcreteModel()

# Decision variables
model.OF = pyo.Var(bounds=(data['OF_lb'], data['OF_ub']))
model.IR = pyo.Var(bounds=(data['IR_lb'], data['IR_ub']))
model.AAR = pyo.Var(bounds=(data['AAR_lb'], data['AAR_ub']))
model.AY = pyo.Var(bounds=(data['AY_lb'], data['AY_ub']))
model.IM = pyo.Var(bounds=(data['IM_lb'], data['IM_ub']))
model.AS = pyo.Var(bounds=(data['AS_lb'], data['AS_ub']))
model.MON = pyo.Var(bounds=(data['MON_lb'], data['MON_ub']))
model.IOR = pyo.Var(bounds=(data['IOR_lb'], data['IOR_ub']))
model.ADF = pyo.Var(bounds=(data['ADF_lb'], data['ADF_ub']))
model.F4 = pyo.Var(bounds=(data['F4_lb'], data['F4_ub']))

# Objective function
model.profit = pyo.Objective(expr=data['p'] * model.AY * model.MON - (data['c_o'] * model.OF + data['c_i'] * model.IM + data['c_r'] * model.IR + data['c_a'] * model.AAR), sense=pyo.maximize)

# Constraints
model.alkylate_yield = pyo.Constraint(expr=model.AY == model.OF * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * (1 + model.alkylate_yield_epsilon))
model.alkylate_yield_epsilon = pyo.Var(bounds=(-0.02, 0.02))

model.motor_octane_number = pyo.Constraint(expr=model.MON == 86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * (model.AS - 89) * (1 + model.motor_octane_number_epsilon))
model.motor_octane_number_epsilon = pyo.Var(bounds=(-0.01, 0.01))

model.acid_dilution_factor = pyo.Constraint(expr=model.ADF == 35.82 - 0.222 * model.F4 * (1 + model.acid_dilution_factor_epsilon))
model.acid_dilution_factor_epsilon = pyo.Var(bounds=(-0.05, 0.05))

model.f4_performance_number = pyo.Constraint(expr=model.F4 == -133 + 3 * model.MON * (1 + model.f4_performance_number_epsilon))
model.f4_performance_number_epsilon = pyo.Var(bounds=(-0.1, 0.1))

model.external_isobutane_to_olefin_ratio = pyo.Constraint(expr=model.IOR == (model.IR + model.IM) / model.OF)

model.acid_strength = pyo.Constraint(expr=model.AS == (93000 * model.AAR) / (model.AAR * model.ADF + 1000 * model.AAR))

model.alkylate_yield_balance = pyo.Constraint(expr=1.22 * model.AY == model.OF + model.IM)

# Solve the model
solver = pyo.SolverFactory('ipopt')
results = solver.solve(model)

# Print results
print(results)
print("Profit:", model.profit())
print("Olefin feed:", model.OF())
print("Isobutane recycle:", model.IR())
print("Acid addition rate:", model.AAR())
print("Alkylate yield:", model.AY())
print("Isobutane makeup:", model.IM())
print("Acid strength:", model.AS())
print("Motor octane number:", model.MON())
print("External isobutane-to-olefin ratio:", model.IOR())
print("Acid dilution factor:", model.ADF())
print("F-4 performance number:", model.F4())

AttributeError: 'ConcreteModel' object has no attribute 'alkylate_yield_epsilon'

## 5. Correct the code to verify model viability (optional)

In [149]:
import pyomo.environ as pyo

# Sample data (replace with actual data if available)
data = {
    'p': 0.65,  # Revenue per barrel of alkylate per octane number
    'c_o': 5.04,  # Cost per barrel of olefin feed
    'c_i': 3.36,  # Cost per barrel of isobutane makeup
    'c_r': 0.035,  # Cost per barrel of isobutane recycle
    'c_a': 10,  # Cost per thousand pounds of acid addition
    'OF_lb': 0,  # Lower bound of olefin feed (barrels per day)
    'OF_ub': 2000,  # Upper bound of olefin feed (barrels per day)
    'IR_lb': 0,   # Lower bound of isobutane recycle (barrels per day)
    'IR_ub': 16000,  # Upper bound of isobutane recycle (barrels per day)
    'AAR_lb': 0,   # Lower bound of acid addition rate (thousands of pounds per day)
    'AAR_ub': 120,  # Upper bound of acid addition rate (thousands of pounds per day)
    'AY_lb': 0,  # Lower bound of alkylate yield (barrels per day)
    'AY_ub': 5000,  # Upper bound of alkylate yield (barrels per day)
    'IM_lb': 0,     # Lower bound of isobutane makeup (barrels per day)
    'IM_ub': 2000,  # Upper bound of isobutane makeup (barrels per day)
    'AS_lb': 85,    # Lower bound of acid strength (weight percent)
    'AS_ub': 93,    # Upper bound of acid strength (weight percent)
    'MON_lb': 90,    # Lower bound of Motor octane number
    'MON_ub': 95,    # Upper bound of Motor octane number
    'IOR_lb': 3,     # Lower bound of External isobutane-to-olefin ratio
    'IOR_ub': 12,     # Upper bound of External isobutane-to-olefin ratio
    'ADF_lb': 1.2,    # Lower bound of Acid dilution factor
    'ADF_ub': 4,    # Upper bound of Acid dilution factor
    'F4_lb': 145,   # Lower bound of F-4 performance number
    'F4_ub': 162,   # Upper bound of F-4 performance number
}

# Model creation
model = pyo.ConcreteModel()

# Decision variables
model.OF = pyo.Var(bounds=(data['OF_lb'], data['OF_ub']))
model.IR = pyo.Var(bounds=(data['IR_lb'], data['IR_ub']))
model.AAR = pyo.Var(bounds=(data['AAR_lb'], data['AAR_ub']))
model.AY = pyo.Var(bounds=(data['AY_lb'], data['AY_ub']))
model.IM = pyo.Var(bounds=(data['IM_lb'], data['IM_ub']))
model.AS = pyo.Var(bounds=(data['AS_lb'], data['AS_ub']))
model.MON = pyo.Var(bounds=(data['MON_lb'], data['MON_ub']))
model.IOR = pyo.Var(bounds=(data['IOR_lb'], data['IOR_ub']))
model.ADF = pyo.Var(bounds=(data['ADF_lb'], data['ADF_ub']))
model.F4 = pyo.Var(bounds=(data['F4_lb'], data['F4_ub']))

# Objective function
model.profit = pyo.Objective(expr=data['p'] * model.AY * model.MON - (data['c_o'] * model.OF + data['c_i'] * model.IM + data['c_r'] * model.IR + data['c_a'] * model.AAR), sense=pyo.maximize)

# Constraints
model.alkylate_yield_epsilon = pyo.Var(bounds=(-0.02, 0.02))
model.alkylate_yield = pyo.Constraint(expr=model.AY == model.OF * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) * (1 + model.alkylate_yield_epsilon))

model.motor_octane_number_epsilon = pyo.Var(bounds=(-0.01, 0.01))
model.motor_octane_number = pyo.Constraint(expr=model.MON == 86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * (model.AS - 89) * (1 + model.motor_octane_number_epsilon))

model.acid_dilution_factor_epsilon = pyo.Var(bounds=(-0.05, 0.05))

model.acid_dilution_factor = pyo.Constraint(expr=model.ADF == 35.82 - 0.222 * model.F4 * (1 + model.acid_dilution_factor_epsilon))
model.f4_performance_number_epsilon = pyo.Var(bounds=(-0.1, 0.1))
model.f4_performance_number = pyo.Constraint(expr=model.F4 == -133 + 3 * model.MON * (1 + model.f4_performance_number_epsilon))


model.external_isobutane_to_olefin_ratio = pyo.Constraint(expr=model.IOR == (model.IR + model.IM) / model.OF)

model.acid_strength = pyo.Constraint(expr=model.AS == (93000 * model.AAR) / (model.AAR * model.ADF + 1000 * model.AAR))

model.alkylate_yield_balance = pyo.Constraint(expr=1.22 * model.AY == model.OF + model.IM)

# Solve the model
solver = pyo.SolverFactory('ipopt')
results = solver.solve(model)

# Print results
print(results)
print("Profit:", model.profit())
print("Olefin feed:", model.OF())
print("Isobutane recycle:", model.IR())
print("Acid addition rate:", model.AAR())
print("Alkylate yield:", model.AY())
print("Isobutane makeup:", model.IM())
print("Acid strength:", model.AS())
print("Motor octane number:", model.MON())
print("External isobutane-to-olefin ratio:", model.IOR())
print("Acid dilution factor:", model.ADF())
print("F-4 performance number:", model.F4())


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 14
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.06062674522399902
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Profit: 159738.90090665472
Olefin feed: 2000.0
Isobutane recycle: 7865.489128949531
Acid addition rate: 0.0
Alkylate yield: 3278.688557376916
Isobutane makeup: 2000.0
Acid strength: 92.88853376059835
Motor octane number: 92.11794990325065
External isobutane-to-olefin ratio: 4.932744525147364
Acid dilution factor: 1.2
F-4 performance number: 154.33049743948308


In [148]:
model.pprint()

14 Var Declarations
    AAR : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :   120 : False : False :  Reals
    ADF : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :   1.2 :   1.2 :     4 : False : False :  Reals
    AS : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :    85 : 92.88853376059836 :    93 : False : False :  Reals
    AY : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 3278.688557376952 :  5000 : False : False :  Reals
    F4 : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :   145 : 154.46141943691325 :   162 : False : False :  Reals
    IM : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 2000.0 :  2000 : False : False

In [146]:
# Alkylation Problem with Couenne
import pyomo.environ as pyo

# Costs
c1, c2, c3, c4, c5 = 0.63, 5.04, 0.035, 10.0, 3.36


# Data
x1_l, x1_u = (0, 2000)
x2_l, x2_u = (0, 16000)
x3_l, x3_u = (0, 120)
x4_l, x4_u = (0, 5000)
x5_l, x5_u = (0, 2000)
x6_l, x6_u = (85, 93)
x7_l, x7_u = (90, 95)
x8_l, x8_u = (3, 12)
x9_l, x9_u = (1.2, 4)
x10_l, x10_u = (145, 162)

# Create a model object
model = pyo.ConcreteModel()

# Declare the variables with bounds (assuming bounds x_i^l and x_i^u are defined elsewhere)
model.x1 = pyo.Var(within=pyo.NonNegativeReals, bounds=(x1_l, x1_u))
model.x2 = pyo.Var(within=pyo.NonNegativeReals, bounds=(x2_l, x2_u))
model.x3 = pyo.Var(within=pyo.NonNegativeReals, bounds=(x3_l, x3_u))
model.x4 = pyo.Var(within=pyo.NonNegativeReals, bounds=(x4_l, x4_u))
model.x5 = pyo.Var(within=pyo.NonNegativeReals, bounds=(x5_l, x5_u))
model.x6 = pyo.Var(within=pyo.NonNegativeReals, bounds=(x6_l, x6_u))
model.x7 = pyo.Var(within=pyo.NonNegativeReals, bounds=(x7_l, x7_u))
model.x8 = pyo.Var(within=pyo.NonNegativeReals, bounds=(x8_l, x8_u))
model.x9 = pyo.Var(within=pyo.NonNegativeReals, bounds=(x9_l, x9_u))
model.x10 = pyo.Var(within=pyo.NonNegativeReals, bounds=(x10_l, x10_u))

# Constants in the objective function and bounds
d4_l, d4_u, d7_l, d7_u, d9_l, d9_u, d10_l, d10_u = 98/100, 102/100, 99/100, 101/100, 95/100, 105/100, 9/10, 11/10

# Objective function
model.objective = pyo.Objective(expr=c1*model.x4*model.x7 - c2*model.x1 - c3*model.x2 - c4*model.x3 - c5*model.x5, sense=pyo.maximize)

# Constraints
model.constraint1 = pyo.Constraint(expr=(model.x1 * (1.12 + 0.13167 * model.x8 - 0.00667 * model.x8**2)) - d4_l * model.x4 >= 0)
model.constraint2 = pyo.Constraint(expr=-(model.x1 * (1.12 + 0.13167 * model.x8 - 0.00667 * model.x8**2)) + d4_u * model.x4 >= 0)
model.constraint3 = pyo.Constraint(expr=(86.35 + 1.098 * model.x8 - 0.038 * model.x8**2 + 0.325 * (model.x6 - 89)) - d7_l * model.x7 >= 0)
model.constraint4 = pyo.Constraint(expr=-(86.35 + 1.098 * model.x8 - 0.038 * model.x8**2 + 0.325 * (model.x6 - 89)) + d7_u * model.x7 >= 0)
model.constraint5 = pyo.Constraint(expr=(35.82 - 0.222 * model.x10) - d9_l * model.x9 >= 0)
model.constraint6 = pyo.Constraint(expr=-(35.82 - 0.222 * model.x10) + d9_u * model.x9 >= 0)
model.constraint7 = pyo.Constraint(expr=(-133 + 3 * model.x7) - d10_l * model.x10 >= 0)
model.constraint8 = pyo.Constraint(expr=-(-133 + 3 * model.x7) + d10_u * model.x10 >= 0)
model.constraint9 = pyo.Constraint(expr=1.22 * model.x4 - model.x1 - model.x5 == 0)
model.constraint10 = pyo.Constraint(expr=((93000 * model.x3)/(model.x3 * model.x9 + 1000 * model.x3)) - model.x6 == 0)
model.constraint11 = pyo.Constraint(expr=((model.x2 + model.x5)/model.x1) - model.x8 == 0)

# Solve the model
solver=pyo.SolverFactory('ipopt')
results = solver.solve(model)

# Display the results
model.pprint()

# Display the results
print("Objective Value:", round(model.objective(), 2))
print("X1:", round(model.x1(), 2))
print("X2:", round(model.x2(), 2))
print("X3:", round(model.x3(), 2))
print("X4:", round(model.x4(), 2))
print("X5:", round(model.x5(), 2))
print("X6:", round(model.x6(), 2))
print("X7:", round(model.x7(), 2))
print("X8:", round(model.x8(), 2))
print("X9:", round(model.x9(), 2))
print("X10:", round(model.x10(), 2))

10 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 2000.0 :  2000 : False : False : NonNegativeReals
    x10 : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :   145 : 155.9409121663327 :   162 : False : False : NonNegativeReals
    x2 : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 10086.916412443237 : 16000 : False : False : NonNegativeReals
    x3 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :   120 : False : False : NonNegativeReals
    x4 : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 3278.688557376949 :  5000 : False : False : NonNegativeReals
    x5 : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
  

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [150]:
print(response.text)

## Mathematical Optimization Model for Alkylation Process

**Parameters:**

* $p$: Revenue per barrel of alkylate per octane number
* $c_o$: Cost per barrel of olefin feed
* $c_i$: Cost per barrel of isobutane makeup
* $c_r$: Cost per barrel of isobutane recycle
* $c_a$: Cost per thousand pounds of acid addition

**Decision Variables:**

* $OF$: Olefin feed (barrels per day)
* $IR$: Isobutane recycle (barrels per day)
* $AAR$: Acid addition rate (thousands of pounds per day)
* $AY$: Alkylate yield (barrels per day)
* $IM$: Isobutane makeup (barrels per day)
* $AS$: Acid strength (weight percent)
* $MON$: Motor octane number
* $IOR$: External isobutane-to-olefin ratio
* $ADF$: Acid dilution factor
* $F4$: F-4 performance number

**Objective Function:**

Maximize daily profit:

$$
\max Z = p \cdot AY \cdot MON - (c_o \cdot OF + c_i \cdot IM + c_r \cdot IR + c_a \cdot AAR)
$$

**Constraints:**

1. **Alkylate yield regression:**
   - $AY = OF \cdot (1.12 + 0.13167 \cdot IOR - 0.00667 \cdot

In [151]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data if available)
data = {
    'p': 2.5,  # Revenue per barrel of alkylate per octane number
    'c_o': 50,  # Cost per barrel of olefin feed
    'c_i': 45,  # Cost per barrel of isobutane makeup
    'c_r': 40,  # Cost per barrel of isobutane recycle
    'c_a': 0.05,  # Cost per thousand pounds of acid addition
    'OF_lb': 1000,  # Lower bound of olefin feed (barrels per day)
    'OF_ub': 5000,  # Upper bound of olefin feed (barrels per day)
    'IR_lb': 500,   # Lower bound of isobutane recycle (barrels per day)
    'IR_ub': 4000,  # Upper bound of isobutane recycle (barrels per day)
    'AAR_lb': 10,   # Lower bound of acid addition rate (thousands of pounds per day)
    'AAR_ub': 100,  # Upper bound of acid addition rate (thousands of pounds per day)
    'AY_lb': 1500,  # Lower bound of alkylate yield (barrels per day)
    'AY_ub': 7000,  # Upper bound of alkylate yield (barrels per day)
    'IM_lb': 0,     